# NBA(미국 농구 리그) 웹사이트에서 다음 경기일정 찾기

In [34]:
from bs4 import BeautifulSoup
import pandas as pd

## Selenium을 활용해 쿠키 팝업창 제거하고 html 저장

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver_path = '/home/rlxdn/Downloads/chromedriver'
driver = webdriver.Chrome(driver_path)
driver.get("https://www.nba.com/schedule?cal=October&region=1")

WebDriverWait(driver, 10).until(
  EC.presence_of_element_located(
      (By.CSS_SELECTOR, "#onetrust-accept-btn-handler")
  )
)

btn = '#onetrust-accept-btn-handler'
driver.find_element_by_css_selector(btn).click()

html = driver.page_source
driver.close()

In [36]:
soup = BeautifulSoup(html)

### select와 반복문을 활용해 경기날짜별 경기의 데이터 가져오기(경기 시간, 경기 지역과 장소, 경기 시간, 경기 팀)

In [37]:
days = soup.select("div.flex.flex-col.pt-5.border-b.border-concrete")
games_list = []

for day in days:
  date = day.find("h4").get_text()

  for i in range(int(day.find("h6").get_text().split(" ")[1])):
    game = day.select("div.py-5.border-t.border-concrete.flex.flex-col")[i]
    gtime = game.select_one("span").text

    for j in range(2):
      team = game.select("a.text-cerulean")[j].text
      if (j+1)%2!=0:
        ateam = team
      else:
        hteam = team

    for k in range(2):
      spot = game.select("div[class='text-sm']")[k].text
      if (k+1)%2==0:
        arena = spot
      else:
        location = spot
        
    games_list.append([date, gtime, ateam, hteam, arena, location])
print(games_list)

[['Sunday, October 3', '3:30 pm ET', 'Brooklyn Nets', 'Los Angeles Lakers', 'Los Angeles,  CA', 'STAPLES Center'], ['Monday, October 4', '7:00 pm ET', 'Philadelphia 76ers', 'Toronto Raptors', 'Toronto,  ON', 'Scotiabank Arena'], ['Monday, October 4', '7:30 pm ET', 'Orlando Magic', 'Boston Celtics', 'Boston,  MA', 'TD Garden'], ['Monday, October 4', '7:30 pm ET', 'Atlanta Hawks', 'Miami Heat', 'Miami,  FL', 'FTX Arena'], ['Monday, October 4', '8:00 pm ET', 'New Orleans Pelicans', 'Minnesota Timberwolves', 'Minneapolis,  MN', 'Target Center'], ['Monday, October 4', '8:00 pm ET', 'Charlotte Hornets', 'Oklahoma City Thunder', 'Oklahoma City,  OK', 'Paycom Center'], ['Monday, October 4', '8:30 pm ET', 'Utah Jazz', 'San Antonio Spurs', 'San Antonio,  TX', 'AT&T Center'], ['Monday, October 4', '10:00 pm ET', 'Golden State Warriors', 'Portland Trail Blazers', 'Portland,  OR', 'Moda Center'], ['Monday, October 4', '10:00 pm ET', 'Phoenix Suns', 'Sacramento Kings', 'Sacramento,  CA', 'Golden 1 C

In [38]:
columns = ['날짜', '경기시간', '홈팀', '어웨이팀', '지역', '아레나']
df = pd.DataFrame(games_list, columns=columns)
df

,날짜,경기시간,홈팀,어웨이팀,지역,아레나
0,"Sunday, October 3",3:30 pm ET,Brooklyn Nets,Los Angeles Lakers,"Los Angeles, CA",STAPLES Center
1,"Monday, October 4",7:00 pm ET,Philadelphia 76ers,Toronto Raptors,"Toronto, ON",Scotiabank Arena
2,"Monday, October 4",7:30 pm ET,Orlando Magic,Boston Celtics,"Boston, MA",TD Garden
3,"Monday, October 4",7:30 pm ET,Atlanta Hawks,Miami Heat,"Miami, FL",FTX Arena
4,"Monday, October 4",8:00 pm ET,New Orleans Pelicans,Minnesota Timberwolves,"Minneapolis, MN",Target Center
...,...,...,...,...,...,...
81,"Friday, October 29",7:30 pm ET,Indiana Pacers,Brooklyn Nets,"Brooklyn, NY",Barclays Center
82,"Saturday, October 30",5:00 pm ET,Boston Celtics,Washington Wizards,"Washington, DC",Capital One Arena
83,"Saturday, October 30",7:00 pm ET,Orlando Magic,Detroit Pistons,"Detroit, MI",Little Caesars Arena
84,"Sunday, October 31",3:30 pm ET,Sacramento Kings,Dallas Mavericks,"Dallas, TX",American Airlines Center


In [39]:
print(f"현지시간 기준 {df['날짜'][0]}일 {df['지역'][0]}의 {df['아레나'][0]}에서 {df['어웨이팀'][0]}, {df['홈팀'][0]}의 경기가 {df['경기시간'][0]}에 예정되어 있습니다.")

현지시간 기준 Sunday, October 3일 Los Angeles,  CA의 STAPLES Center에서 Los Angeles Lakers, Brooklyn Nets의 경기가 3:30 pm ET에 예정되어 있습니다.
